# Wildfire Mitigation using Quantum Annealing
## Hadamard Hooligans

### Grid Generation

In [293]:
import random 
from collections import deque
from IPython.display import display, HTML
from math import sqrt
import json

def clamp(i, imin, imax):
    return (i < imin)*imin + (imin <= i < imax)*i + (imax <= i)*imax

def get_neighbors(x, y, n):
    directions = [(-1,0), (1,0), (0,-1), (0,1)]
    return [(x+dx, y+dy) for dx, dy in directions if 0 <= x+dx < n and 0 <= y+dy < n]

def valid_types_around(grid, x, y, n):
    neighbors = get_neighbors(x, y, n)
    types = set(grid[nx][ny]['type'] for nx, ny in neighbors if grid[nx][ny] is not None)
    possible = []

    if types and types.issubset({'E', 'H'}):
        possible.append('H')
    if types & {'H'}:
        possible.append('E')
    if types & {'E', 'D'} and 'H' not in types:
        possible.append('D')

    return possible

def generate_grid(n, d_limit=3):
    grid = [[None for _ in range(n)] for _ in range(n)]
    start_x, start_y = random.randint(0, n-1), random.randint(0, n-1)
    grid[start_x][start_y] = {
        'type': 'E',
        'population': 0,
        'risk': 0,
    }
    frontier = deque([(start_x, start_y)])

    while frontier:
        x, y = frontier.popleft()
        for nx, ny in get_neighbors(x, y, n):
            if grid[nx][ny] is None:
                options = valid_types_around(grid, nx, ny, n)
                if options:
                    typ = random.choice(options)
                    pop = random.randint(500, 4000) if typ == 'H' else 0
                    risk = 0

                    # grid[nx][ny] = choice
                    grid[nx][ny] = {
                        'type': typ,
                        'population': pop,
                        'risk': risk
                    }
                    frontier.append((nx, ny))

    return grid

def print_grid(grid):
    html = "<table style='border-collapse: collapse;'>"
    
    for row in grid:
        html += "<tr style='text-align: center; line-height: 20px; font-size: 15pt;'>"
        for cell in row:
            ty = cell['type']
            if ty in ['D', 'H', 'E', 'F']:
                if ty == 'D':
                    color = 'darkgreen'
                elif ty == 'H':
                    color = 'gray'
                elif ty == 'E':
                    color = 'green'
                elif ty == 'F':
                    color = 'orange'
                pop = '' if ty == 'F' else cell['population']
                html += "<td style='background-color: {}; color: white; padding: 10px; border: 1px solid black; width: 60px;'><b>{}</b><br>{}<br>{}</td>".format(color, cell['type'], pop, cell['risk'])  # Dark Green for D
            else:
                html += "<td style='background-color: white; color: black; padding: 10px; border: 1px solid black; width: 60px;'>.</td>"  # Default for empty cells
            
        html += "</tr>"
    
    html += "</table>"
    
    display(HTML(html))

def is_adjacent(grid, r, c, typ):
    # top
    r2 = max(0, r - 1)
    if grid[r2][c]['type'] == typ:
        return True
    # right
    c2 = min(c + 1, n - 1)
    if grid[r][c2]['type'] == typ:
        return True
    # bottom
    r2 = min(r + 1, n - 1)
    if grid[r2][c]['type'] == typ:
        return True
    # left
    c2 = max(0, c - 1)
    if grid[r][c2]['type'] == typ:
        return True
    return False

# Example usage
n = 10
grid = generate_grid(n)

rad = 3
region = list(range(-rad, rad+1))

# fire center
fire_center = (random.randint(0, n - 1), random.randint(0, n - 1))

while grid[fire_center[0]][fire_center[1]]['type'] != 'D':
    fire_center = (random.randint(0, n - 1), random.randint(0, n - 1))
grid[fire_center[0]][fire_center[1]]['type'] = 'F'
grid[fire_center[0]][fire_center[1]]['risk'] = 1

# fire spread
prob_spread = 1
dist = 1

while dist < n and prob_spread > 0:
    i, j = fire_center
    
    # top
    if 0 <= i - dist and random.random() <= prob_spread and grid[i - dist][j]['type'] == 'D':
        grid[i - dist][j]['type'] = 'F'
        grid[i - dist][j]['risk'] = 1

    # right
    if j + dist < n and random.random() <= prob_spread and grid[i][j + dist]['type'] == 'D':
        grid[i][j + dist]['type'] = 'F'
        grid[i][j + dist]['risk'] = 1
        
    # bottom
    if i + dist < n and random.random() <= prob_spread and grid[i + dist][j]['type'] == 'D':
        grid[i + dist][j]['type'] = 'F'
        grid[i + dist][j]['risk'] = 1

    # left
    if 0 <= j - dist and random.random() <= prob_spread and grid[i][j - dist]['type'] == 'D':
        grid[i][j - dist]['type'] = 'F'
        grid[i][j - dist]['risk'] = 1

    # top-right
    if random.random() <= prob_spread and (0 <= i - dist and j + dist < n and grid[i - dist][j + dist]['type'] == 'D'):
        grid[i - dist][j + dist]['type'] = 'F'
        grid[i - dist][j + dist]['risk'] = 1

    # bottom-right
    if random.random() <= prob_spread and (i + dist < n and j + dist < n and grid[i + dist][j + dist]['type'] == 'D'):
        grid[i + dist][j + dist]['type'] = 'F'
        grid[i + dist][j + dist]['risk'] = 1
        
    # bottom-left
    if random.random() <= prob_spread and (i + dist < n and 0 <= j - dist and grid[i + dist][j - dist]['type'] == 'D'):
        grid[i + dist][j - dist]['type'] = 'F'
        grid[i + dist][j - dist]['risk'] = 1

    # top-left
    if random.random() <= prob_spread and (0 <= i - dist and 0 <= j - dist and grid[i - dist][j - dist]['type'] == 'D'):
        grid[i - dist][j - dist]['type'] = 'F'
        grid[i - dist][j - dist]['risk'] = 1

    dist += 1
    prob_spread -= 0.1

# risk fall off
for r in range(n):
    for c in range(n):
        if grid[r][c]['type'] != 'F' and is_adjacent(grid, r, c, 'F'):
            grid[r][c]['risk'] = 0.75

risk_rad = 2
risk_region = list(range(-risk_rad,risk_rad+1))

for i in range(n):
    for j in range(n):
        risk_tot = 0
        for di in risk_region:
            for dj in risk_region:
                ni, nj = clamp(i+di, 0, n-1), clamp(j + dj, 0, n-1)
                risk_tot += (grid[ni][nj]['risk'])/sqrt(0.01+di**2+dj**2)
                
        grid[i][j]['risk'] = round(risk_tot / len(risk_region)**2,1)

f = json.dumps(grid)
with open('grid.json', 'w') as file:
    file.write(f)

print_grid(grid)
with open('grid.json', 'r') as grid_file:
    loaded_grid = json.loads(grid_file.read())

E00.0,H15110.0,H27710.0,H10560.0,H7530.0,E00.0,H14370.1,E00.3,D00.7,F0.9
E00.0,H11450.0,H38830.0,H21100.0,H12220.1,H13740.1,H25590.2,E00.6,D00.7,F0.9
H29040.0,E00.1,H32750.1,H24450.2,E00.2,H26190.2,E00.6,F0.8,D00.7,F0.9
H38130.1,H33820.2,E00.5,E00.5,D00.6,E00.3,D00.6,D00.7,F0.9,F0.9
H38080.1,E00.4,F0.6,F0.6,F0.7,D00.6,F0.7,F0.8,F0.9,F0.9
E00.1,D00.1,D00.4,D00.5,D00.5,D00.3,D00.6,D00.7,F0.9,F0.9
D00.0,D00.1,D00.1,D00.1,D00.2,D00.3,D00.6,F0.8,D00.7,F0.9
D00.0,D00.0,D00.1,D00.1,D00.2,D00.5,F0.7,D00.6,D00.6,F0.7
D00.0,D00.0,D00.1,D00.2,D00.5,D00.3,D00.5,D00.2,D00.2,D00.5
D00.0,D00.0,D00.1,D00.5,F0.6,D00.5,D00.2,D00.1,D00.1,D00.1


### QUBO

In [294]:
import numpy as np
from pyqubo import Binary

# Define grid and region labels
indices = range(n)
regions = ['a', 'b', 'g']

burn_rad = 1
burn_region = list(range(-burn_rad,burn_rad+1))

def cost(i, j, r):
    rad = 2
    region = list(range(-rad, rad+1))

    # 1000000000000000 is an arbitrary large number to prevent it from doing silly things like putting out a fire that doesn't exist
    bigNum = 1000000000000000
    type = loaded_grid[i][j]['type']
    if r == 'a': # Air support
        base_cost = bigNum - (bigNum-1500000)*(type == 'F') # $1500000, cost of air support for square mile
        additional_discount = 0
        for di in region:
            for dj in region:
                ni, nj = clamp(i+di, 0, n-1), clamp(j + dj, 0, n-1)
                neighbor = loaded_grid[ni][nj]
                additional_discount += (20*(neighbor['type'] == 'D') + (neighbor['type'] == 'F'))*250000/sqrt(0.01+di**2+dj**2) # best used in deep forest and fires   
        return base_cost - additional_discount
        
    elif r == 'g':# Ground crew
        base_cost = bigNum - (bigNum-300000)*(type == 'F') # $300000, cost of ground crew for square mile, best used on fire
        additional_discount = 0
        for di in region:
            for dj in region:
                ni, nj = clamp(i+di, 0, n-1), clamp(j + dj, 0, n-1)
                neighbor = loaded_grid[ni][nj]
                additional_discount += ((neighbor['type'] == 'E')*200000000)/sqrt(0.01+di**2+dj**2) # best used close to houses and edge forest
        return base_cost - additional_discount

    elif r == 'b': # Fire lines with controlled burns
        base_cost = bigNum - (bigNum-64000)*(type == 'E') # $64000, cost of ground crew for mile long, best near houses
        additional_discount = 0
        for di in burn_region:
            for dj in burn_region:
                if di == 0 and dj == 0:
                    continue
                ni, nj = clamp(i+di, 0, n-1), clamp(j + dj, 0, n-1)
                neighbor = loaded_grid[ni][nj]
                additional_discount += (neighbor['type'] == 'F')*2200/sqrt(0.01+di**2+dj**2) # best used on fires
        return base_cost - additional_discount

In [ ]:
def w(i, j, r):
    global region
    
    benefit = 0 # how much worth of lives are you saving
    for di in region:
        for dj in region:
            ni, nj = clamp(i+di, 0, n-1), clamp(j + dj, 0, n-1)
            cell = loaded_grid[ni][nj]
            benefit -= cell['risk']*cell['population']*75000000/sqrt(0.01+ni**2+nj**2) # 2020 FEMA statistical value of life: 7.5 million
    return benefit + cost(i, j, r)


# Constraint penalty strength
lambda_penalty = 9000000 # we have a small penalty for sending the same type of support to the same area, as the more support you send, you have diminishing gains

# Create binary variables x_ijr and build the Hamiltonian
variables = []
x_vars = {}
objective = 0
constraints = 0

for i in indices:
    for j in indices:
        terms = []
        for r in regions:
            name = f"x_{i}{j}{r}"
            x = Binary(name)
            x_vars[name] = x
            variables.append(name)
            objective += w(i, j, r) * x
            terms.append(x)
        # Add constraint: at most one resource per (i,j)
        sum_terms = sum(terms)
        constraints += (sum_terms * sum_terms)  # squared penalty

# Total Hamiltonian = objective - reward + penalty
H = objective + lambda_penalty * constraints

# Compile and convert to QUBO
model = H.compile()
qubo, offset = model.to_qubo()

# Map variable names to indices
index = {var: i for i, var in enumerate(variables)}
size = len(variables)

# Initialize QUBO matrix
matrix = np.zeros((size, size))

# Fill the matrix
for (u, v), coeff in qubo.items():
    i, j = index[u], index[v]
    matrix[i][j] += coeff

# Print matrix
print("QUBO Matrix:")
header = "        " + "".join(f"{v:>8}" for v in variables)
print(header)
for i, row in enumerate(matrix):
    row_str = "".join(f"{val:8.1f}" for val in row)
    print(f"{variables[i]:>7} |{row_str}")

print("\nOffset:", offset)


QUBO Matrix:
           x_00a   x_00b   x_00g   x_01a   x_01b   x_01g   x_02a   x_02b   x_02g   x_03a   x_03b   x_03g   x_04a   x_04b   x_04g   x_05a   x_05b   x_05g   x_06a   x_06b   x_06g   x_07a   x_07b   x_07g   x_08a   x_08b   x_08g   x_09a   x_09b   x_09g   x_10a   x_10b   x_10g   x_11a   x_11b   x_11g   x_12a   x_12b   x_12g   x_13a   x_13b   x_13g   x_14a   x_14b   x_14g   x_15a   x_15b   x_15g   x_16a   x_16b   x_16g   x_17a   x_17b   x_17g   x_18a   x_18b   x_18g   x_19a   x_19b   x_19g   x_20a   x_20b   x_20g   x_21a   x_21b   x_21g   x_22a   x_22b   x_22g   x_23a   x_23b   x_23g   x_24a   x_24b   x_24g   x_25a   x_25b   x_25g   x_26a   x_26b   x_26g   x_27a   x_27b   x_27g   x_28a   x_28b   x_28g   x_29a   x_29b   x_29g   x_30a   x_30b   x_30g   x_31a   x_31b   x_31g   x_32a   x_32b   x_32g   x_33a   x_33b   x_33g   x_34a   x_34b   x_34g   x_35a   x_35b   x_35g   x_36a   x_36b   x_36g   x_37a   x_37b   x_37g   x_38a   x_38b   x_38g   x_39a   x_39b   x_39g   x_40a   x_40b   

### Quantum Annealing

In [296]:
import neal

# We will use the QUBO we defined in the previous python cell.

# Initialize the simulated annealing sampler
sampler = neal.SimulatedAnnealingSampler()

# Solve the QUBO problem using the sampler with a specified number of reads.
# The more reads, the better the chance to find a lower energy solution.
sampleset = sampler.sample_qubo(qubo, num_reads=100)

# Extract and display the best solution found
best_solution = sampleset.first.sample
best_energy = sampleset.first.energy

print("Best Solution:")
print(best_solution)
print("Energy of the Best Solution:")
print(best_energy)

arr = []

html = "<table style='border-collapse: collapse;'>"

for r, row in enumerate(grid):
    html += "<tr style='text-align: center; line-height: 20px; font-size: 15pt;'>"
    for c, cell in enumerate(row):
        ty = cell['type']
        if ty in ['D', 'H', 'E', 'F']:
            if ty == 'D':
                color = 'darkgreen'
            elif ty == 'H':
                color = 'gray'
            elif ty == 'E':
                color = 'green'
            elif ty == 'F':
                color = 'orange'
            
            # Get a, b, g from best_solution
            a = best_solution.get(f'x_{r}{c}a', 0)
            b = best_solution.get(f'x_{r}{c}b', 0)
            g = best_solution.get(f'x_{r}{c}g', 0)
            
            abg = ''
            if a: abg += '✈️'
            if b: abg += '🔥'
            if g: abg += '🧑‍🚒'
            
            # Use abg instead of population
            html += "<td style='background-color: {}; color: white; padding: 10px; border: 1px solid black; width: 60px;'><b>{}</b><br>{}<br>{}</td>".format(color, cell['type'], abg, cell['risk'])
        else:
            html += "<td style='background-color: white; color: black; padding: 10px; border: 1px solid black; width: 60px;'>.</td>"  # Default for empty cells
    html += "</tr>"

html += "</table>"

display(HTML(html))


Best Solution:
{'x_00a': np.int8(0), 'x_00b': np.int8(1), 'x_00g': np.int8(0), 'x_01a': np.int8(0), 'x_01b': np.int8(0), 'x_01g': np.int8(0), 'x_02a': np.int8(0), 'x_02b': np.int8(0), 'x_02g': np.int8(0), 'x_03a': np.int8(0), 'x_03b': np.int8(0), 'x_03g': np.int8(0), 'x_04a': np.int8(0), 'x_04b': np.int8(0), 'x_04g': np.int8(0), 'x_05a': np.int8(0), 'x_05b': np.int8(1), 'x_05g': np.int8(0), 'x_06a': np.int8(0), 'x_06b': np.int8(0), 'x_06g': np.int8(0), 'x_07a': np.int8(0), 'x_07b': np.int8(1), 'x_07g': np.int8(0), 'x_08a': np.int8(0), 'x_08b': np.int8(0), 'x_08g': np.int8(0), 'x_09a': np.int8(1), 'x_09b': np.int8(0), 'x_09g': np.int8(1), 'x_10a': np.int8(0), 'x_10b': np.int8(1), 'x_10g': np.int8(0), 'x_11a': np.int8(0), 'x_11b': np.int8(0), 'x_11g': np.int8(0), 'x_12a': np.int8(0), 'x_12b': np.int8(0), 'x_12g': np.int8(0), 'x_13a': np.int8(0), 'x_13b': np.int8(0), 'x_13g': np.int8(0), 'x_14a': np.int8(0), 'x_14b': np.int8(0), 'x_14g': np.int8(0), 'x_15a': np.int8(0), 'x_15b': np.int8(0

E🔥0.0,H0.0,H0.0,H0.0,H0.0,E🔥0.0,H0.1,E🔥0.3,D0.7,F✈️🧑‍🚒0.9
E🔥0.0,H0.0,H0.0,H0.0,H0.1,H0.1,H0.2,E🔥0.6,D0.7,F✈️🧑‍🚒0.9
H0.0,E🔥0.1,H0.1,H0.2,E🔥0.2,H0.2,E🔥0.6,F✈️🧑‍🚒0.8,D0.7,F✈️🧑‍🚒0.9
H0.1,H0.2,E🔥0.5,E🔥0.5,D0.6,E🔥0.3,D0.6,D0.7,F✈️🧑‍🚒0.9,F✈️🧑‍🚒0.9
H0.1,E🔥0.4,F✈️🧑‍🚒0.6,F✈️🧑‍🚒0.6,F✈️🧑‍🚒0.7,D0.6,F✈️🧑‍🚒0.7,F✈️🧑‍🚒0.8,F✈️🧑‍🚒0.9,F✈️🧑‍🚒0.9
E🔥0.1,D0.1,D0.4,D0.5,D0.5,D0.3,D0.6,D0.7,F✈️🧑‍🚒0.9,F✈️0.9
D0.0,D0.1,D0.1,D0.1,D0.2,D0.3,D0.6,F✈️0.8,D0.7,F✈️0.9
D0.0,D0.0,D0.1,D0.1,D0.2,D0.5,F✈️0.7,D0.6,D0.6,F✈️0.7
D0.0,D0.0,D0.1,D0.2,D0.5,D0.3,D0.5,D0.2,D0.2,D0.5
D0.0,D0.0,D0.1,D0.5,F✈️0.6,D0.5,D0.2,D0.1,D0.1,D0.1
